# Orchestration Multi-Agents : Pipeline de Finalisation de Notebook

Bienvenue dans ce notebook de démonstration qui illustre comment plusieurs **agents IA** peuvent collaborer afin de finaliser automatiquement un notebook Jupyter.  

L’orchestration repose sur trois agents clés :

- **CoderAgent** : Examine et modifie le contenu du notebook (cellules de code ou Markdown).
- **ReviewerAgent** : Exécute le notebook, détecte d’éventuelles anomalies et demande des corrections.
- **AdminAgent** : Valide ou rejette la version finale du notebook.

### Objectif général

Nous souhaitons montrer la capacité de ces agents à :

1. Analyser et améliorer un notebook existant (ou fraîchement généré).
2. Tester et vérifier que l’exécution se déroule sans erreur.
3. Accepter ou refuser certaines modifications pour aboutir à un notebook **validé**.

---

Les sections suivantes détaillent la configuration et le fonctionnement pas à pas de l’orchestration, jusqu’à l’approbation finale par l’AdminAgent.


## 1. Installation des dépendances

Pour ce pipeline, nous utilisons notamment :

- **papermill** : exécuter des notebooks et injecter des paramètres.
- **nbformat** : manipuler la structure interne d’un notebook Jupyter.
- **ipywidgets** : créer des widgets interactifs (par exemple, un bouton d’upload).
- **semantic-kernel** : orchestrer et faire collaborer les agents conversationnels.

> **Remarque** : Après installation, il peut être nécessaire de **redémarrer le kernel** pour que ces paquets soient disponibles.


In [ ]:
# Cellule Code : Installation des packages requis
# ------------------------------------------------
%pip install papermill nbformat ipywidgets semantic-kernel --quiet


## 2. Import des bibliothèques et configuration

Dans cette cellule, nous allons :

- Importer des modules Python classiques (`os`, `json`, `hashlib`, `logging`, etc.).
- Importer `nbformat` et `papermill` pour la lecture, l’écriture et l’exécution de notebooks.
- Importer `random` et `datetime` pour divers utilitaires (tirage aléatoire de tâche, timestamps...).
- Importer les composants de **semantic-kernel** pour interagir avec nos agents conversationnels.
- Configurer un logger coloré pour faciliter le suivi et le débogage des étapes d’orchestration.

Le logger nous aidera à comprendre les actions qui se déroulent à chaque instant.


In [ ]:
# Cellule Code : Imports et configuration du logger
# -------------------------------------------------
import os
import json
import hashlib
import logging
import nbformat
import papermill as pm
import random
from datetime import datetime

# Imports liés à Semantic Kernel (agents, connecteurs, etc.)
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions.kernel_arguments import KernelArguments

# Configuration d'un logger avec mise en couleur
class ColorFormatter(logging.Formatter):
    colors = {
        'DEBUG': '\033[94m',
        'INFO': '\033[92m',
        'WARNING': '\033[93m',
        'ERROR': '\033[91m',
        'CRITICAL': '\033[91m\033[1m'
    }
    reset = '\033[0m'

    def format(self, record):
        msg = super().format(record)
        return f"{self.colors.get(record.levelname, '')}{msg}{self.reset}"

logger = logging.getLogger("Orchestration")
logger.setLevel(logging.DEBUG)

if not logger.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    formatter = ColorFormatter(
        "%(asctime)s [%(levelname)s] %(name)s - %(message)s",
        datefmt="%H:%M:%S"
    )
    ch.setFormatter(formatter)
    logger.addHandler(ch)

logger.info("Configuration initiale terminée.")


## 3. Classe `NotebookState` : gestion de l’état du notebook

La classe `NotebookState` gère la vie du notebook cible :

- **Chargement / Sauvegarde** : lecture et écriture depuis/vers un fichier `.ipynb`.
- **Exécution** (via Papermill) pour détecter les erreurs.
- **Gestion des états** : `specified`, `implemented`, `tested`, `validated`.
- **Mises à jour ciblées** : possibilité de remplacer la source d’une cellule précise.

Cette classe centralise toutes les opérations, afin que les agents puissent aisément lire/modifier le notebook au fur et à mesure.


In [ ]:
# Cellule Code : Classe NotebookState
# -----------------------------------
class NotebookState:
    """
    Gère le statut et le contenu d'un notebook.
    Les 4 états possibles sont : 'specified', 'implemented', 'tested', 'validated'.
    """

    def __init__(self, notebook_path: str):
        self.notebook_path = notebook_path
        self._cached_notebook = None
        self._status = "specified"
        self._load_notebook()

    def _load_notebook(self):
        """Charge le notebook depuis le chemin spécifié."""
        if not os.path.exists(self.notebook_path):
            raise FileNotFoundError(f"Notebook introuvable: {self.notebook_path}")

        with open(self.notebook_path, "r", encoding="utf-8") as f:
            self._cached_notebook = nbformat.read(f, as_version=4)

        logger.debug(
            f"[NotebookState] Notebook '{self.notebook_path}' chargé, "
            f"nb de cellules={len(self._cached_notebook.cells)}."
        )

    def get_notebook_json(self) -> str:
        """Retourne une représentation JSON du notebook en mémoire."""
        return json.dumps(self._cached_notebook, indent=2, ensure_ascii=False)

    def log_notebook_state(self, max_length: int = 10000):
        """Log une partie du contenu JSON du notebook, à titre de diagnostic."""
        notebook_json = self.get_notebook_json()
        snippet = notebook_json[:max_length]
        if len(notebook_json) > max_length:
            snippet += ">>> TRUNCATED <<<"
        logger.debug(f"[NotebookState] Current notebook state (truncated):\n{snippet}")

    def get_status(self) -> str:
        """Renvoie l'état courant du notebook."""
        return self._status

    def set_status(self, new_status: str):
        """Met à jour l'état du notebook et log la transition."""
        logger.info(f"[NotebookState] Passage de l'état {self._status} → {new_status}")
        self._status = new_status

    def save_notebook(self, path: str = ""):
        """Enregistre le notebook sur disque (par défaut au même chemin)."""
        if not path:
            path = self.notebook_path
        try:
            with open(path, "w", encoding="utf-8") as f:
                nbformat.write(self._cached_notebook, f)
            logger.info(f"[NotebookState] Notebook sauvegardé sous {path}")
        except Exception as e:
            logger.error(f"[save_notebook] Erreur de sauvegarde: {e}")
            raise

    def execute_notebook(self) -> bool:
        """
        Exécute le notebook via Papermill et retourne True si tout se passe bien,
        ou False si une exception est levée pendant l’exécution.
        """
        logger.info(f"[NotebookState] Exécution Papermill sur {self.notebook_path}.")
        # 1. Sauvegarde le notebook avant exécution
        self.save_notebook()

        success = True
        try:
            pm.execute_notebook(
                input_path=self.notebook_path,
                output_path=self.notebook_path,
                kernel_name="python3",
                progress_bar=False,
                log_output=True
            )
        except Exception as e:
            # Papermill lève une exception si une cellule a planté
            logger.error(f"[execute_notebook] Erreur lors de l'exécution: {e}")
            success = False
        finally:
            # 3. Recharger la version (qui inclut désormais les outputs)
            self._load_notebook()
            # 4. Re-sauvegarder pour inclure toutes les sorties dans le notebook
            self.save_notebook()
            logger.info("[NotebookState] Notebook mis à jour après exécution (avec sorties).")

        return success

    def update_cell(self, cell_index: int, new_source: str):
        """
        Modifie la source d’une cellule donnée, puis sauvegarde.
        """
        old_src = self._cached_notebook.cells[cell_index].source
        self._cached_notebook.cells[cell_index].source = new_source
        logger.info(f"[NotebookState] Mise à jour de la cellule {cell_index}")
        logger.debug(f"[NotebookState] Ancien contenu:\n{old_src}")
        logger.debug(f"[NotebookState] Nouveau contenu:\n{new_source}")
        self.save_notebook()

    def find_cell_indices_by_content(self, content_pattern: str):
        """Retourne la liste d'indices de cellules contenant `content_pattern`."""
        indices = []
        for i, c in enumerate(self._cached_notebook.cells):
            if content_pattern in c.source:
                indices.append(i)
        return indices

    def is_approved(self) -> bool:
        """Renvoie True si l'état du notebook est 'validated'."""
        return self._status == "validated"


## 4. (Optionnel) Téléverser votre propre notebook

Cette étape vous permet de **remplacer** le template par votre propre fichier `.ipynb`.  
Si vous téléversez un fichier, exécutez la cellule suivante pour l’enregistrer, sinon le pipeline continuera avec le *template* par défaut.


In [ ]:
# Cellule Code : Widget pour uploader un notebook (optionnel)
# ----------------------------------------------------------
from IPython.display import display
import ipywidgets as widgets

uploader = widgets.FileUpload(
    accept='.ipynb',
    multiple=False,
    description='Choisir notebook',
    disabled=False
)
display(uploader)


## 5. Création (ou chargement) du Notebook cible + Test de validité

Cette cellule :
1. Copie un *template* interne **ou** récupère le notebook `.ipynb` téléversé.
2. Sélectionne une **description de tâche** (au hasard parmi plusieurs exemples).
3. Exécute automatiquement le notebook via Papermill pour détecter d’éventuelles erreurs.
4. Vérifie l’intégrité et confirme l’exécutabilité du notebook.

Ainsi, nous nous assurons que tout est correctement initialisé avant d’entamer l’orchestration.


In [ ]:
# Cellule Code : Initialisation du notebook ciblé et injection de la tâche
# -----------------------------------------------------------------------
import shutil


POSSIBLE_TASKS = [
    # 1) Notebook d'analyse de données simulées (bureautique)
    "Créer un notebook Python qui génère aléatoirement un DataFrame de ventes mensuelles (12 mois), affiche des graphiques d'évolution et exporte un rapport PDF.",

    # 2) Notebook d'utilitaires : compression/décompression
    "Créer un notebook Python qui crée un dossier local avec quelques fichiers, puis compresse ce dossier en ZIP, et vérifie la taille et l'intégrité après décompression.",

    # 3) Notebook de jeu (exemple : Snake en mode démo)
    "Créer un notebook Python qui implémente un mini jeu console (Snake ou Pong) en mode 'demo' et se termine après un certain nombre de 'ticks'.",

    # 4) Notebook d’automatisation bureautique (Excel)
    "Créer un notebook Python utilisant openpyxl (ou xlsxwriter) pour générer deux fichiers Excel puis les fusionner avec un résumé global.",

    # 5) Notebook de mini web scraping (flux RSS)
    "Créer un notebook Python qui télécharge un flux RSS public (p.ex. CNN), stocke les titres dans un CSV, puis génère un nuage de mots (WordCloud).",

    # 6) Notebook de visualisation de données (plotly)
    "Créer un notebook Python qui requête DBpedia (SPARQL) et affiche un graphique final (Plotly).",

    # 7) Notebook de machine learning (classification Titanic)
    "Créer un notebook Python qui charge le dataset Titanic depuis une URL, effectue une analyse basique et un court modèle de classification. Le dataset est accessible à l'url https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv",

    # 8) Notebook de machine learning 2 (classification IRIS)
    "Construire un notebook scikit-learn sur le dataset IRIS et réaliser un court modèle de classification. Le dataset est accessible à l'url https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv"
]


def apply_task_description(notebook_state: NotebookState, task_description: str) -> bool:
    """
    Remplace le placeholder {{TASK_DESCRIPTION}} dans la cellule Markdown
    contenant le titre '## 0. Objectif du Notebook'.
    """
    for idx, cell in enumerate(notebook_state._cached_notebook.cells):
        if cell.cell_type == "markdown" and "## 0. Objectif du Notebook" in cell.source:
            new_source = cell.source.replace(
                "{{TASK_DESCRIPTION}}",
                f"<!-- TASK-ID: {hash(task_description)} -->\n{task_description}"
            )
            notebook_state.update_cell(idx, new_source)
            return True
    return False

SOURCE_NOTEBOOK = "Notebook-Template.ipynb"
DEST_NOTEBOOK = "Notebook-Generated.ipynb"

# 1) Soit on utilise le fichier uploadé, soit on copie le template.
if uploader.value:
    fichier = uploader.value[0]
    DEST_NOTEBOOK = fichier.name
    with open(DEST_NOTEBOOK, 'wb') as f:
        f.write(fichier.content)
    logger.info(f"Notebook uploadé : {DEST_NOTEBOOK}")
else:
    shutil.copy2(SOURCE_NOTEBOOK, DEST_NOTEBOOK)
    logger.info(f"Création depuis le template : {DEST_NOTEBOOK}")

# 2) Instancier NotebookState sur le fichier choisi (ou copié).
notebook_state = NotebookState(DEST_NOTEBOOK)

# 3) Injecter une description de tâche aléatoire dans le notebook.
chosen_task = random.choice(POSSIBLE_TASKS)
changed = apply_task_description(notebook_state, chosen_task)

# 4) Exécuter le notebook pour s’assurer qu’il tourne sans erreur.
if changed:
    logger.info("Placeholder {{TASK_DESCRIPTION}} remplacé avec succès.")
    logger.info("Exécution du notebook pour valider...")
    notebook_state.execute_notebook()
    logger.info("Notebook ré-exécuté après injection de la tâche.")
else:
    logger.warning("Aucun placeholder détecté : vérifiez la cellule Markdown contenant {{TASK_DESCRIPTION}}.")


## 6. Architecture de plugins : extension du NotebookState

Nous définissons plusieurs **plugins** pour interagir avec l’instance `NotebookState` :

- **BaseNotebookPlugin** : expose le contenu du notebook via `get_notebook_content()`.
- **CoderNotebookPlugin** : modifie le notebook (`update_cell_by_content()`) et signale la fin d’implémentation (`finish_implementation()`).
- **ReviewerNotebookPlugin** : exécute le notebook et l’approuve (ou non) via `validate_notebook(approve=...)`.
- **AdminNotebookPlugin** : validation finale via `approve_notebook(admin_ok=...)`.

Chacun de ces plugins est déclaré comme **fonction** (`@kernel_function`) utilisable par nos agents conversationnels.


In [ ]:
# ================================
# Plugins actualisés
# ================================

class BaseNotebookPlugin:
    """Plugin de base pour manipuler NotebookState."""
    def __init__(self, state: NotebookState):
        self.state = state

    @kernel_function(
        name="get_notebook_content",
        description="Renvoie le notebook (format JSON) révisé actuellement."
    )
    def get_notebook_content(self) -> str:
        logger.info("[BaseNotebookPlugin] get_notebook_content()")
        return self.state.get_notebook_json()


class NotebookEditingMixin:
    """
    Mixin exposant la fonction d'édition de cellule via
    'update_cell_by_content'. Le code sera partagé entre
    le Coder et l’Admin.
    """

    def update_cell_anyway(self, pattern: str, new_source: str) -> str:
        """Logique commune pour mettre à jour une cellule."""
        indices = self.state.find_cell_indices_by_content(pattern)
        if len(indices) == 0:
            return f"Aucune cellule ne contient le motif '{pattern}'."
        if len(indices) > 1:
            return f"Plusieurs cellules contiennent '{pattern}', soyez plus précis."

        old_status = self.state.get_status()
        # Autoriser la modification quel que soit l'état, mais on peut imposer des règles
        # (ex. si on souhaite refuser l'édition quand le notebook est déjà validé).
        if old_status == "validated":
            return "Édition impossible, le notebook est déjà validé."

        self.state.update_cell(indices[0], new_source)
        return f"Cellule contenant '{pattern}' mise à jour (état actuel={old_status})."


class CoderNotebookPlugin(BaseNotebookPlugin, NotebookEditingMixin):
    """
    Plugin pour l'agent 'Coder'.
    Hérite de BaseNotebookPlugin et NotebookEditingMixin,
    donc il peut lire + éditer + terminer l'implémentation.
    """

    @kernel_function(
        name="update_cell_by_content",
        description="Modifie la première cellule contenant 'content_pattern'."
    )
    def update_cell_by_content(self, content_pattern: str, new_source: str) -> str:
        logger.info("[CoderNotebookPlugin] update_cell_by_content()")
        status = self.state.get_status()
        if status not in ["specified", "implemented"]:
            return f"Erreur: l'état actuel est '{status}' => plus de modifications autorisées."

        return self.update_cell_anyway(content_pattern, new_source)

    @kernel_function(
        name="finish_implementation",
        description="Déclare le notebook 'implemented' lorsque le code est prêt."
    )
    def finish_implementation(self) -> str:
        logger.info("[CoderNotebookPlugin] finish_implementation()")
        status = self.state.get_status()
        if status == "specified":
            self.state.set_status("implemented")
            return "Le notebook passe à l'état 'implemented'."
        elif status == "implemented":
            return "Le notebook est déjà en état 'implemented'."
        else:
            return f"Impossible de passer en 'implemented' depuis l'état '{status}'."


class ReviewerNotebookPlugin(BaseNotebookPlugin):
    """
    Plugin pour l'agent 'Reviewer'. Ne peut pas éditer,
    mais exécute et approuve ou non.
    """

    @kernel_function(
        name="validate_notebook",
        description="Exécute le notebook et approuve ou non (approve=True/False)."
    )
    def validate_notebook(self, approve: bool = True) -> str:
        logger.info(f"[ReviewerNotebookPlugin] validate_notebook(approve={approve})")
        status = self.state.get_status()
        if status != "implemented":
            return f"Le reviewer ne peut pas valider, état actuel = '{status}'."

        success = self.state.execute_notebook()
        if not success:
            # Si échec => retour à 'specified'
            self.state.set_status("specified")
            return ("Erreur d'exécution dans le notebook (voir logs). "
                    "Retour à l'état 'specified' pour corrections.")

        if approve:
            self.state.set_status("tested")
            return "Le reviewer approuve => état 'tested'."
        else:
            self.state.set_status("specified")
            return "Le reviewer refuse => retour à 'specified'."


class AdminNotebookPlugin(BaseNotebookPlugin, NotebookEditingMixin):
    """
    Plugin pour l'agent 'Admin'.
    Peut lire, éditer et approuver ou non. Après plusieurs
    éditions, il peut repasser en 'specified' pour rendre
    la main au Coder.
    """

    @kernel_function(
        name="update_cell_by_content",
        description="Modifie la première cellule correspondant au pattern. Permet de clarifier ou approfondir."
    )
    def admin_edit_cell_by_content(self, content_pattern: str, new_source: str) -> str:
        logger.info("[AdminNotebookPlugin] admin_edit_cell_by_content()")
        # L'admin a le droit d'éditer même si l'état est 'tested'
        # ou autre. Mais on peut imposer un revert d'état si on veut
        # exiger un nouveau cycle de relecture.
        msg = self.update_cell_anyway(content_pattern, new_source)
        # Si besoin, repasser en 'specified' pour déclencher un nouveau cycle
        # par ex. on le fait systématiquement ici, ou seulement après X edits.
        # Ex. reprenons l'idée "après chaque edit => revert en 'specified'".
        self.state.set_status("specified")
        return msg + " -> Revert du notebook à l'état 'specified'."

    @kernel_function(
        name="approve_notebook",
        description="Validation finale: admin_ok=True => 'validated', sinon 'specified'."
    )
    def approve_notebook(self, admin_ok: bool = True) -> str:
        logger.info(f"[AdminNotebookPlugin] approve_notebook(admin_ok={admin_ok})")
        status = self.state.get_status()
        if status != "tested":
            return f"Impossible d'approuver: l'état est '{status}' (attendu: 'tested')."

        if admin_ok:
            self.state.set_status("validated")
            return "Notebook validé => état 'validated'."
        else:
            self.state.set_status("specified")
            return "Admin refuse => retour à l'état 'specified'."


## 7. Stratégies d’orchestration

Deux stratégies clés :

1. **ApprovedBasedTerminationStrategy**  
   - Met fin au dialogue si le notebook est validé ou si le nombre d’itérations max est atteint.

2. **NotebookAwareSelectionStrategy**  
   - Sélectionne l’agent en fonction de l’état du notebook :  
     - `specified` ⇒ **CoderAgent**  
     - `implemented` ⇒ **ReviewerAgent**  
     - `tested` ⇒ **AdminAgent**  
   - N’appelle plus personne si l’état est `validated`.

Ainsi, chaque agent n’intervient que lorsqu’il est requis.


## Faire démarrer le processus entre les mains de l’Admin

Nous souhaitons que la **première intervention** soit celle de l’Admin, afin de réviser/améliorer les specs dans le Markdown. Pour cela, nous modifions notre `NotebookAwareSelectionStrategy` :

- Une variable interne `has_first_agent_run = False` nous permet de repérer la **toute première** sélection.
- Si le notebook est `specified` **et** qu’aucun agent n’a encore parlé, on renvoie **AdminAgent** d’emblée.
- Ensuite, quand l’Admin fait sa première édition, l’état repasse en `specified`, ce qui permettra au **Coder** de prendre la main lors du second passage. 
- On conserve le fonctionnement normal pour les phases `implemented` (Reviewer) et `tested` (Admin).


In [ ]:
# Cellule Code : Définition des stratégies
# ----------------------------------------
from pydantic import PrivateAttr

class ApprovedBasedTerminationStrategy(TerminationStrategy):
    _state: NotebookState = PrivateAttr()
    _max_steps: int = PrivateAttr(default=10)

    def __init__(self, state: NotebookState, max_steps: int = 20):
        super().__init__()
        self._state = state
        self._max_steps = max_steps
        self._current_step = 0

    async def should_agent_terminate(self, agent, history):
        self._current_step += 1
        is_approved = self._state.is_approved()
        logger.debug(
            f"[TerminationStrategy] Step={self._current_step}/{self._max_steps}, "
            f"IsApproved={is_approved}"
        )
        if is_approved:
            logger.info("[TerminationStrategy] Notebook approuvé => arrêt.")
            return True
        if self._current_step >= self._max_steps:
            logger.warning(f"[TerminationStrategy] max_steps={self._max_steps} atteint => arrêt.")
            return True
        return False


class NotebookAwareSelectionStrategy(SelectionStrategy):
    def __init__(self, state: NotebookState):
        super().__init__()
        self._state = state
        self._index = -1
        self._has_first_agent_run = False  # Ajout : pour repérer la 1ère sélection

    def reset(self) -> None:
        self._index = -1
        self._has_first_agent_run = False

    async def select_agent(self, agents, history):
        current_status = self._state.get_status()
        logger.debug(f"[SelectionStrategy] nb_agents={len(agents)}, statut={current_status}, first_run={not self._has_first_agent_run}")

        if current_status == "validated":
            logger.info("[SelectionStrategy] Notebook déjà validé => fin.")
            return None

        # Récupérer nos agents
        coder = next((a for a in agents if a.name == "CoderAgent"), None)
        reviewer = next((a for a in agents if a.name == "ReviewerAgent"), None)
        admin = next((a for a in agents if a.name == "AdminAgent"), None)

        # 1) Cas particulier : on est en 'specified' et aucune intervention encore => Admin d'abord
        if not self._has_first_agent_run and current_status == "specified":
            if admin:
                self._has_first_agent_run = True
                logger.info("Première intervention : AdminAgent (pour réviser le Markdown avant le coder).")
                return admin

        # Cas normal, après la 1ère intervention
        if current_status == "specified" and coder:
            to_return = coder
        elif current_status == "implemented" and reviewer:
            to_return = reviewer
        elif current_status == "tested" and admin:
            to_return = admin
        else:
            logger.warning(f"[SelectionStrategy] Aucun agent approprié pour état='{current_status}' => stop.")
            to_return = None

        if to_return and not self._has_first_agent_run:
            self._has_first_agent_run = True

        if to_return:
            logger.info(f"Prochain agent sélectionné : {to_return.name}")
        return to_return


## 8. Création des 3 agents (Coder, Reviewer, Admin)

Nous paramétrons ici trois **agents** :

1. **CoderAgent**  
   - Peut lire le notebook en JSON et le modifier (code ou Markdown).  
   - Déclare la fin de l’implémentation lorsque le code est prêt.
2. **ReviewerAgent**  
   - Consulte le notebook, exécute, approuve (`approve=True`) ou refuse (`approve=False`).  
   - En cas d’échec, on retourne en phase de modifications.
3. **AdminAgent**  
   - Consulte le notebook en état `tested`.  
   - Valide en définitive (`admin_ok=True`) ou retourne à `specified` si non satisfaisant.


In [ ]:
# Cellule Code : Création des 3 agents et configuration avec prise en compte du .env
# ---------------------------------------------------------------------------------
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI

# Charger d'éventuelles variables d'environnement (avant tout usage)
load_dotenv()

def create_chat_completion_service(service_id: str = "default"):
    """
    Crée une instance de ChatCompletion (OpenAI) en fonction
    des variables d'environnement et du mode choisi.

    Variables .env possibles :
      - GLOBAL_LLM_SERVICE  : "OpenAI" (par défaut).
      - OPENAI_API_KEY      : Clé API à utiliser (par défaut, "sk-fake").
      - OPENAI_CHAT_MODEL_ID: Nom du modèle (ex: "gpt-4", "gpt-4o-mini", etc.).
      - OPENAI_BASE_URL     : URL personnalisée pour un endpoint compatible OpenAI.
    """

    # Lecture des variables .env
    global_llm_service     = os.getenv("GLOBAL_LLM_SERVICE", "OpenAI").strip()
    openai_api_key         = os.getenv("OPENAI_API_KEY", "sk-fake").strip()
    openai_chat_model_id   = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-4o-mini").strip()
    openai_base_url        = os.getenv("OPENAI_BASE_URL", "").strip()

    # Selon la valeur de GLOBAL_LLM_SERVICE, soit on utilise la version officielle OpenAI,
    # soit on prend un endpoint personnalisé (LLM local, Ollama, vLLM, LMStudio, etc.)
    if openai_base_url.lower() == "":
        logger.info("Utilisation du service OpenAI officiel.")
        return OpenAIChatCompletion(
            service_id=service_id,
            ai_model_id=openai_chat_model_id,
            api_key=openai_api_key
        )
    else:
        logger.info(f"Utilisation d'un endpoint compatible OpenAI : {openai_base_url}")
        # Client "asynchrone" pour requêter un endpoint local ou distant
        openAIClient = AsyncOpenAI(
            api_key=openai_api_key,
            base_url=openai_base_url
        )
        return OpenAIChatCompletion(
            service_id=service_id,
            ai_model_id=openai_chat_model_id,
            async_client=openAIClient
        )

def create_kernel_for_agent(agent_id: str, plugin_instance) -> Kernel:
    """
    Instancie un Kernel SemanticKernel pour l'agent donné,
    en y ajoutant le plugin associé (Coder, Reviewer ou Admin).
    Utilise create_chat_completion_service() pour choisir le backend OpenAI
    (officiel ou endpoint custom) en fonction du .env.
    """
    k = Kernel()

    # Instancier le service de ChatCompletion en fonction du .env
    chat_service = create_chat_completion_service(service_id="default")

    # Ajouter ce service au kernel
    k.add_service(chat_service)

    # Ajouter le plugin de l'agent (Coder, Reviewer, Admin)
    k.add_plugin(plugin_instance, plugin_name=f"{agent_id}_plugin")

    return k

# Instancier les plugins (basés sur NotebookState)
coder_plugin = CoderNotebookPlugin(notebook_state)
reviewer_plugin = ReviewerNotebookPlugin(notebook_state)
admin_plugin = AdminNotebookPlugin(notebook_state)

# Créer un kernel dédié par agent (en tenant compte du .env)
coder_kernel = create_kernel_for_agent("coder_kernel", coder_plugin)
reviewer_kernel = create_kernel_for_agent("reviewer_kernel", reviewer_plugin)
admin_kernel = create_kernel_for_agent("admin_kernel", admin_plugin)

# Configurer l'auto-appel de fonctions
coder_settings = coder_kernel.get_prompt_execution_settings_from_service_id("default")
coder_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

reviewer_settings = reviewer_kernel.get_prompt_execution_settings_from_service_id("default")
reviewer_settings.function_choice_behavior = FunctionChoiceBehavior.Required()

admin_settings = admin_kernel.get_prompt_execution_settings_from_service_id("default")
admin_settings.function_choice_behavior = FunctionChoiceBehavior.Required()

# Définition des 3 agents conversationnels
coder_agent = ChatCompletionAgent(
    service_id="coder_kernel",
    kernel=coder_kernel,
    name="CoderAgent",
    instructions=(
        "Vous êtes le **Coder**. Votre rôle : Implémenter avec le plus grand soin les instructions établies dans les cellules de markdown du notebook\n"
        "1) Visualisez systématiquement le JSON du notebook via get_notebook_content().\n"
        "2) Analyser et modifier le code/Markdown si nécessaire (update_cell_by_content...).\n"
        "3) Préserver les marqueurs importants.\n"
        "4) Quand c'est prêt, appelez finish_implementation().\n"
        "5) Si besoin, faire de nouvelles itérations.\n"
    ),
    arguments=KernelArguments(settings=coder_settings)
)

reviewer_agent = ChatCompletionAgent(
    service_id="reviewer_kernel",
    kernel=reviewer_kernel,
    name="ReviewerAgent",
    instructions=(
        "Vous êtes le **Reviewer**. Votre rôle: vérifier avec le plus grand soin le travail du codeur, l'aiguiller pour d'éventuelles corrections, et valider son travail quand il est parfait.\n"
        "1) Commencez toujours par consulter le notebook complet (get_notebook_content()) après chaque mise à jour.\n"
        "2) Appelez validate_notebook(approve=True/False) selon la qualité.\n"
        "   - True => passage à 'tested'\n"
        "   - False => retour à 'specified'\n"
        "3) Vérifiez scrupuleusement les éventuelles erreurs et la bonne implémentation dans les cellules de code des notions présentées dans les cellules de Markdown.\n"
    ),
    arguments=KernelArguments(settings=reviewer_settings)
)



admin_agent = ChatCompletionAgent(
    service_id="admin_kernel",
    kernel=admin_kernel,
    name="AdminAgent",
    instructions=(
        "Vous êtes l'**Admin**. Vous êtes en charge de la réalisation du notebook et faite preuve d'un grand niveau d'exigence avec votre équipe.\n"
        "1) **Vous intervenez au début** de la conversation pour spécifier au mieux le notebook, "
        "   en particulier le Markdown (admin_edit_cell_by_content). "
        "   Corrigez, reformulez, enrichissez les explications.\n"
        "2) Après chaque édition de votre part, l'agent codeur est solicité pour l'implémentation des cellules de code.\n"
        "3) Quand le notebook est en 'tested', c'est à dire que le reviewer a validé les éditions du codeur, vous pouvez à nouveau l'éditer "
        "   ou valider via approve_notebook(admin_ok=True/False).\n"
        "4) Visez un niveau d'exactitude et de clarté élevé, tant pour le code, les sorties de cellule, que pour la documentation.\n"
        "5) Une fois satisfait, validez définitivement (admin_ok=True) "
        "   pour passer à 'validated'.\n"
    ),
    arguments=KernelArguments(settings=admin_settings)
)



# Mettre en place la stratégie de sélection & la stratégie d'arrêt
termination_strategy = ApprovedBasedTerminationStrategy(notebook_state)
selection_strategy = NotebookAwareSelectionStrategy(notebook_state)

# GroupChat pour orchestrer la conversation
group_chat = AgentGroupChat(
    agents=[coder_agent, reviewer_agent, admin_agent],
    selection_strategy=selection_strategy,
    termination_strategy=termination_strategy
)

logger.info("Agents créés et group_chat initialisé avec instructions mises à jour.")


## 9. Boucle de conversation

Nous lançons la **conversation multi-agents** :

1. À chaque itération, un agent (choisi selon l’état du notebook) produit une réponse.
2. Les agents peuvent appeler les fonctions `update_cell_by_content`, `validate_notebook`, `approve_notebook`, etc.
3. La boucle s’arrête lorsque :
   - Le notebook est validé (`is_approved() == True`), ou
   - Le nombre maximal de tours est dépassé.

Vous verrez dans les logs : le choix de l’agent, les appels de fonction, les retours d’exécution, etc.


In [ ]:
# Cellule Code : Lancement de la conversation async
# ------------------------------------------------
import asyncio

async def run_conversation():
    """
    Lance la conversation multi-agents pour finaliser 
    le notebook selon l'état du pipeline.
    """
    try:
        logger.info("Version initiale du notebook :")
        notebook_state.log_notebook_state()

        # Contexte initial (historique de conversation)
        initial_content = notebook_state.get_notebook_json()
        group_chat.history.add_system_message(f"NOTEBOOK CONTENT:\n{initial_content}")
        group_chat.history.add_user_message(
            "Bonjour, j'aimerais finaliser ce notebook et corriger ses cellules si besoin."
        )
        logger.info("=== Début de la conversation entre agents ===")
        iteration = 0
        async for message in group_chat.invoke():
            iteration += 1
            role = message.name
            content = message.content
            logger.info(f"[STEP {iteration} - {role}] {content}")

            # Arrêter si le notebook est validé
            if notebook_state.is_approved():
                logger.info("Notebook approuvé => fin de la conversation.")
                break

        logger.info("Version finale du notebook, après la conversation :")
        notebook_state.log_notebook_state(max_length=20000)

    except Exception as e:
        logger.error(f"Erreur inattendue: {str(e)}")
    finally:
        logger.info(f"Statut final - Approuvé: {notebook_state.is_approved()}")
        logger.info("=== Fin de la conversation ===")

await run_conversation()
